In [ ]:
import numpy as np
import glob
import cv2
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline 

## Camera calibration, distortion and warping functions

In [ ]:
def camera_calibration(img_size, verbose = False):
    
    # Number of corners in the checkboard
    nx = 9
    ny = 6

    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        if verbose:
            print('Calibration image ' + fname + ': ' + str(ret))

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (8,6), corners, ret)
            #write_name = 'corners_found'+str(idx)+'.jpg'
            #cv2.imwrite(write_name, img)
            #cv2.imshow('img', img)

            if verbose:
                f, ax = plt.subplots()
                ax.imshow(img)
    
    # Compute calibration coefficients
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    
    # Save for future use
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump( dist_pickle, open( "camera_cal/calibration.p", "wb" ) )
    
    return mtx, dist


def undistort_image(img, mtx, dist):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst


def warp_image(img):
    
    #           1280          720
    img_size = (img.shape[1], img.shape[0])
    h = img.shape[0]
    w = img.shape[1]
    
    src = np.float32([[575+5, 460],
                      [w-575, 460],
                      [260, 685],
                      [w-260+20, 685]])
    
    side = 300
    dst = np.float32([[side, 0],
                      [w-side, 0],
                      [side, h],
                      [w-side, h]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped, M, Minv


def region_of_interest(img):
    
    # Parameters of the region of interest
    height = img.shape[0]
    width = img.shape[1]
    roi_h_center = width / 2
    roi_v_center = 420
    roi_flat_size = width / 20
    roi_v_side_left = 0
    roi_v_side_right = roi_v_side_left
    roi_bottom = height
    
    # Vertices of the polygon describing the region of interest
    vertices = np.array([[(0,roi_bottom),
                          (0, height - roi_v_side_left),
                          (roi_h_center-roi_flat_size/2, roi_v_center),
                          (roi_h_center+roi_flat_size/2, roi_v_center),
                          (width, height - roi_v_side_right),
                          (width,roi_bottom)]],
                        dtype=np.int32)
    
    # Defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # Filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    # Returning the image only where mask pixels are nonzero
    return cv2.bitwise_and(img, mask)

## Image processing functions

In [ ]:
def gradient_magnitude_threshold(img, sobel_kernel=3, thresh=(0, 255)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobel = np.sqrt(sobelx**2 + sobely**2)
    
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary_output


def gradient_direction_threshold(img, sobel_kernel=3, thresh_deg=(0., 90.)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    dir_sobel = np.arctan2(sobely, sobelx)
    
    thresh = (thresh_deg[0] / 180. * np.pi, thresh_deg[1] / 180. * np.pi)
    binary_output = np.zeros_like(dir_sobel)
    binary_output[(dir_sobel >= thresh[0]) & (dir_sobel <= thresh[1])] = 1
    
    blur_kernel = sobel_kernel
    blurred = cv2.GaussianBlur(binary_output, (blur_kernel, blur_kernel), 0)
    
    return blurred


def gradient_x_threshold(img, sobel_kernel=3, thresh=(0, 255)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return binary_output


def color_select(img):
    
    rgb_r = img[:,:,0]
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    hls_h = hls[:,:,0]
    hls_l = hls[:,:,1]
    hls_s = hls[:,:,2]
    
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    lab_b = lab[:,:,2]
    
    ret, rgb_r_binary = cv2.threshold(rgb_r, 210, 255, cv2.THRESH_BINARY)
    ret, hls_s_binary = cv2.threshold(hls_s, 175, 255, cv2.THRESH_BINARY)
    ret, hls_l_binary = cv2.threshold(hls_l, 200, 255, cv2.THRESH_BINARY)
    hls_l_binary2 = cv2.adaptiveThreshold(hls_l,
                                          255,
                                          cv2.ADAPTIVE_THRESH_MEAN_C,
                                          cv2.THRESH_BINARY,
                                          31,
                                          -40)
    ret, lab_b_binary = cv2.threshold(lab_b, 160, 255, cv2.THRESH_BINARY)
    lab_b_binary2 = cv2.adaptiveThreshold(lab_b,
                                          255,
                                          cv2.ADAPTIVE_THRESH_MEAN_C,
                                          cv2.THRESH_BINARY,
                                          21,  # it's working for the first 2 videos with 15, but 25 looks better
                                          -6)

    
    # Combine
    combined_binary = np.zeros_like(hls_l_binary)
    """
    combined_binary[(hls_s_binary == 255) |
                    (hls_l_binary == 255) |
                    (lab_b_binary == 255)] = 1
    """
    """
    combined_binary[(hls_l_binary == 255) |
                    (lab_b_binary == 255)] = 1
    """
    combined_binary[(hls_l_binary == 255) |
                    (lab_b_binary2 == 255)] = 1
    
    # Debug
    if 0:
        if 0:
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
            ax1.imshow(lab_b, cmap='gray')
            ax1.set_title('lab_b')
            ax2.imshow(lab_b_binary, cmap='gray')
            
        if 1:
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
            ax1.imshow(lab_b, cmap='gray')
            ax1.set_title('lab_b2')
            ax2.imshow(lab_b_binary2, cmap='gray')
            
        if 0:
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
            ax1.imshow(hls_l, cmap='gray')
            ax1.set_title('hls_l')
            ax2.imshow(hls_l_binary, cmap='gray')
            
        if 0:
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
            ax1.imshow(hls_l, cmap='gray')
            ax1.set_title('hls_l2')
            ax2.imshow(hls_l_binary2, cmap='gray')
            
        if 0:
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
            ax1.imshow(hls_s, cmap='gray')
            ax1.set_title('hls_s')
            ax2.imshow(hls_s_binary, cmap='gray')
            
        if 0:
            f, ax = plt.subplots(1, 1, figsize=(20,10))
            ax.imshow(combined_binary, cmap='gray')
            ax.set_title('combined')
            
    return combined_binary


def image_processing_pipeline(img, mtx, dist):
    
    # Make a local copy
    img = np.copy(img)
    
    # Produce the undistorted version of the original image. This goes directly to the output.
    undistorted = undistort_image(img, mtx, dist)
    
    # Process the image to highlight lines
    gradient_binary = gradient_x_threshold(img, sobel_kernel=9, thresh=(20, 100))
    color_binary = color_select(img)
    
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    stacked_binary = np.dstack((np.zeros_like(gradient_binary),
                                gradient_binary,
                                color_binary))

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(color_binary)
    """
    combined_binary[(gradient_binary == 1) |
                    (color_binary == 1)] = 1
    """
    combined_binary[(color_binary == 1)] = 1
    
    # Region of interest
    # region = region_of_interest(combined_binary)
    
    # Warping
    processed, M, Minv = warp_image(undistort_image(combined_binary, mtx, dist))
    
    # Debug
    if 0:
        print(stacked_binary.shape)
        print(combined_binary.shape)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Stacked')
        ax1.imshow(stacked_binary)
        ax2.set_title('Combined')
        ax2.imshow(combined_binary, cmap='gray')
    
    return undistorted, processed, M, Minv

## Line detection functions

In [ ]:
def lines_fit_new_frame(img, search_area='half', margin=80, show=False):
    
    # Histogram of the bottom half of the image (where less noisy line data are)
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    quarterpoint = np.int(midpoint/2)
    if search_area == 'half':
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    elif search_area == 'quarter':
        leftx_base = np.argmax(histogram[quarterpoint:midpoint]) + quarterpoint
        rightx_base = np.argmax(histogram[midpoint:(midpoint+quarterpoint)]) + midpoint
    
    # Number of sliding windows (vertical)
    nwindows = 10
    
    # Set height of windows
    window_height = np.int(img.shape[0] / nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
        
    # Set minimum number of pixels found to recenter window
    minpix = 50
    
    # Empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Rectangle for visualization
    rectangles = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Append the data for the current rectangle. The actual drawing can be done later on if needed.
        rectangles.append((win_y_low, win_y_high,
                           win_xleft_low, win_xleft_high,
                           win_xright_low, win_xright_high))
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit, right_fit = (None, None)
    if len(leftx) is not 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) is not 0:
        right_fit = np.polyfit(righty, rightx, 2)
        
    left_linetype, right_linetype = detect_linetype(leftx, rightx)
    
    # Show results
    if show:
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((img, img, img))*255
        
        # New figure and axis
        f, ax = plt.subplots()
        
        # Generate x and y values for plotting
        ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
        # Draw the windows on the visualization image
        for rectangle in rectangles:
            cv2.rectangle(out_img,
                          (rectangle[2],rectangle[0]),
                          (rectangle[3],rectangle[1]),
                          (0,255,0),2) 
            cv2.rectangle(out_img,
                          (rectangle[4],rectangle[0]),
                          (rectangle[5],rectangle[1]),
                          (0,255,0), 2)
        

        # Left line pixels in red and right line pixels in blue
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        
        # Plot image
        plt.imshow(out_img)
        
        # Fitted lines
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        
    return left_fit, right_fit, left_lane_inds, right_lane_inds, left_linetype, right_linetype


def lines_fit_based_on_previous_frame(img, prev_left_fit, prev_right_fit, margin=80, show=False):

    # Assume you now have a new warped binary image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    left_lane_inds = ((nonzerox > (prev_left_fit[0]*(nonzeroy**2) + prev_left_fit[1]*nonzeroy + prev_left_fit[2] - margin)) & 
                      (nonzerox < (prev_left_fit[0]*(nonzeroy**2) + prev_left_fit[1]*nonzeroy + prev_left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (prev_right_fit[0]*(nonzeroy**2) + prev_right_fit[1]*nonzeroy + prev_right_fit[2] - margin)) & 
                       (nonzerox < (prev_right_fit[0]*(nonzeroy**2) + prev_right_fit[1]*nonzeroy + prev_right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Fit a second order polynomial to each
    left_fit, right_fit = (None, None)
    if len(leftx) != 0:
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
        
    left_linetype, right_linetype = detect_linetype(leftx, rightx)
        
    # Show results
    if show:
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((img, img, img))*255
        window_img = np.zeros_like(out_img)
        
        # Generate x and y values for plotting
        ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
        # Left line pixels in red and right line pixels in blue
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area (based on the old fit)
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        plt.imshow(result)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        
    return left_fit, right_fit, left_lane_inds, right_lane_inds, left_linetype, right_linetype


def detect_linetype(leftx, rightx, balance_thresh=1.4, continuous_thresh=8000):
    # Detect which line is continuous and which not
    left_npoints = np.float(leftx.shape[0])
    right_npoints = np.float(rightx.shape[0])
    if right_npoints == 0:
        balance = 100
    else:
        balance = left_npoints / right_npoints
    
    # 0 = dashed
    # 1 = continuous
    if balance > balance_thresh:
        return 1., 0.
    elif balance < 1/balance_thresh:
        return 0., 1.
    else:
        if (left_npoints + right_npoints) / 2 > continuous_thresh:
            return 1., 1.
        else:
            return 0., 0.
        

def curvature_radius_and_distance_from_centre(img, left_fit, right_fit,
                                                   left_lane_inds, right_lane_inds):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension, lane line is 10 ft = 3.048 meters
    xm_per_pix = 3.7/700 # meters per pixel in x dimension, lane width is 12 ft = 3.7 meters
    
    # Default values
    left_curverad, right_curverad, center_dist = (0, 0, 0)
    
    # It's important where you want to measure the radius.
    # In this case, I chose the bottom of the image
    curvature_h = img.shape[0]
    ploty = np.linspace(0, curvature_h-1, curvature_h)
    y_eval = np.max(ploty)
  
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Default output values
    left_curve_radius = None
    right_curve_radius = None
    centre_dist = None
    
    if len(leftx) != 0 and len(rightx) != 0:
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
        
        # Calculate the new radii of curvature in meters
        left_curve_radius = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curve_radius = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
    # Distance from center is image x midpoint - mean of l_fit and r_fit intercepts 
    if left_fit is not None and right_fit is not None:
        car_position = img.shape[1]/2  # based on camera
        left_fit_x_int = left_fit[0]*curvature_h**2 + left_fit[1]*curvature_h + left_fit[2]
        right_fit_x_int = right_fit[0]*curvature_h**2 + right_fit[1]*curvature_h + right_fit[2]
        lane_centre_position = (right_fit_x_int + left_fit_x_int) /2
        centre_dist = (car_position - lane_centre_position) * xm_per_pix
        
    return left_curve_radius, right_curve_radius, centre_dist


def draw_on_image(original_img, binary_img,
                  left_fit, right_fit,
                  left_lane_inds, right_lane_inds,
                  Minv,
                  left_curve_radius=None,
                  right_curve_radius=None,
                  centre_dist=None,
                  left_linetype=None,
                  right_linetype=None,
                  show_processed=True,
                  show=False):
    
    # Make a copy of the original image
    original_img_copy = np.copy(original_img)
    if left_fit is None or right_fit is None:
        return original_img_copy
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    cv2.polylines(color_warp, np.int32([pts_left]), isClosed=False, color=(255,0,255), thickness=25)
    cv2.polylines(color_warp, np.int32([pts_right]), isClosed=False, color=(0,255,255), thickness=25)
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (original_img_copy.shape[1], original_img_copy.shape[0])) 
    
    # Combine the result with the original image
    original_image_with_lines = cv2.addWeighted(original_img_copy, 1, newwarp, 0.3, 0)
    
    # Add curvature radius. The continuous line has more weight than the dashed one.
    font = cv2.FONT_HERSHEY_DUPLEX
    if left_curve_radius is not None and right_curve_radius is not None:
        if left_linetype > 0.5 and right_linetype < 0.5:
            weights = [10.0, 1.0]
        elif left_linetype < 0.5 and right_linetype > 0.5:
            weights = [1.0, 10.0]
        else:
            weights = [1.0, 1.0]
        curve_radius = np.average([left_curve_radius, right_curve_radius], weights=weights)
        
        text = 'Radius: ' + '{:04.2f}'.format(curve_radius) + 'm'
        cv2.putText(original_image_with_lines, text, (40,70), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
    
    # Add distance from centre, if provided
    if centre_dist is not None:
        if centre_dist > 0:
            direction = 'right'
        else:
            direction = 'left'
        text = '{:04.2f}'.format(abs(centre_dist)) + 'm ' + direction + ' of centre'
        cv2.putText(original_image_with_lines, text, (40,120), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
    
    if 0:
        if left_linetype is not None and right_linetype is not None:
            text = 'L: {}  R: {}'.format(left_linetype, right_linetype)
            cv2.putText(original_image_with_lines, text, (40,170), font, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
    # Show processed image in a box on the upper right corner
    if show_processed:
        
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_img, binary_img, binary_img))*255
        window_img = np.zeros_like(out_img)
        
        # Left line pixels in red and right line pixels in blue
        nonzero = binary_img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        
        # Draw fit line
        margin = 5
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))
        
        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (255, 255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (255, 255, 0))
        little_img = cv2.addWeighted(out_img, 1, window_img, 1., 0)
        
        little_img = cv2.resize(little_img, None,
                                      fx=0.25, fy=0.25,
                                      interpolation = cv2.INTER_CUBIC)
        original_image_with_lines[40:40+little_img.shape[0], \
                                  original_image_with_lines.shape[1] - little_img.shape[1] - 40 : \
                                  original_image_with_lines.shape[1] - 40] = little_img
    
    if show:
        f, ax = plt.subplots()
        ax.imshow(original_image_with_lines)
        
    return original_image_with_lines


# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.fits_history = []#[np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #number of detected pixels
        # self.px_count = None
        
        # line type (0=dashed, 1=continuous)
        self.best_linetype = 0.
        self.linetypes_history = []
           
        # Maximum memory depth
        self.N = 10
    
    def add_fit(self, fit, inds, linetype):
        
        # The last iteration produced a fit
        if fit is not None:
            
            # We already have a best fit in memory
            if self.best_fit is not None:
                self.diffs = abs(fit-self.best_fit)
                
            # The new fit is way too different from the best fit we have in memory,
            # we'll do as if we had no fit. However, an exception 
            if (self.diffs[0] > 0.01 or self.diffs[1] > 1.0 or self.diffs[2] > 1000.0) and \
               len(self.fits_history) > 0:
                # We haven't detected a fit
                self.detected = False
                
                # We have a list of old fits
                if len(self.fits_history) > 0:
                    # We do with what we have
                    self.best_fit = np.average(self.fits_history, axis=0)
            
            # The new fit is not too different from the best fit we have in memory
            else:
                # We detected a fit
                self.detected = True
                # Append the fit to the list of fits
                self.fits_history.append(fit)
                if len(self.fits_history) > self.N:
                    # Remove last element
                    self.fits_history = self.fits_history[len(self.fits_history)-self.N:]
                # Compute the best fit using the new one
                self.best_fit = np.average(self.fits_history, axis=0)
            
            ## Line type
            # Append the line type to the list of line types
            self.linetypes_history.append(linetype)
            if len(self.linetypes_history) > self.N:
                # Remove last element
                self.linetypes_history = self.linetypes_history[len(self.linetypes_history)-self.N:]
            # Compute the best fit using the new one
            self.best_linetype = np.round(np.average(self.linetypes_history, axis=0))
        
        # The last iteration didn't produce a fit, we need to live with what we have
        else:
            # We haven't detected a fit
            self.detected = False
            
            # We don't want the list to become too old, otherwise the line can get stuck in some position
            # and never recover from there (e.g. challenge video)
            if len(self.fits_history) > 0:
                self.fits_history = self.fits_history[:len(self.fits_history)-1]
            
            # We have a list of old fits
            if len(self.fits_history) > 0:
                # We do with what we have
                self.best_fit = np.average(self.fits_history, axis=0)
                    

## Pipeline

In [ ]:
def pipeline(img):
    
    image = img.copy()
    
    # Image processing
    undistorted, processed, M, Minv = image_processing_pipeline(image, mtx, dist)
    
    # Decide whether to work on a new frame or use info from the previous frame
    if not left_line.detected or not right_line.detected:
        left_fit, \
        right_fit, \
        left_lane_inds, \
        right_lane_inds, \
        left_linetype, \
        right_linetype = lines_fit_new_frame(processed,
                                             show=False)
    else:
        left_fit, \
        right_fit, \
        left_lane_inds, \
        right_lane_inds, \
        left_linetype, \
        right_linetype = lines_fit_based_on_previous_frame(processed,
                                                           prev_left_fit=left_line.best_fit,
                                                           prev_right_fit=right_line.best_fit,
                                                           show=False)
    
    # Now we have a fit, check conditions to decide whether this fit makes sense or not.
    # If it does, add it to the list of good fits in the left_line and right_line objects, else, just
    # say no good fit comes from this frame and let the object decide which of the previous ones makes
    # sense to use
    
    # TODO: for the moment, every new fit is a good fit, worth object's attention
    left_line.add_fit(left_fit, left_lane_inds, left_linetype)
    right_line.add_fit(right_fit, right_lane_inds, right_linetype)
    
    # Use the best fit, if it's the current one or some previous, is the object's decision
    if left_line.best_fit is not None and right_line.best_fit is not None:
        
        # Compute curvature and distance from centre
        left_curve_radius, \
        right_curve_radius, \
        centre_dist = curvature_radius_and_distance_from_centre(processed,
                                                                left_line.best_fit,
                                                                right_line.best_fit,
                                                                left_lane_inds,        # possible error?
                                                                right_lane_inds)       # possible error?
        
        # Produce output
        undistorted_with_lines = draw_on_image(undistorted, processed,
                                               left_line.best_fit,
                                               right_line.best_fit,
                                               left_lane_inds,
                                               right_lane_inds,
                                               Minv,
                                               left_curve_radius=left_curve_radius,
                                               right_curve_radius=right_curve_radius,
                                               centre_dist=centre_dist,
                                               left_linetype=left_line.best_linetype,
                                               right_linetype=right_line.best_linetype,
                                               show_processed=True,
                                               show=False)
        
        return undistorted_with_lines
    
    else:
        return undistorted

## Use example

https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/2b62a1c3-e151-4a0e-b6b6-e424fa46ceab/lessons/40ec78ee-fb7c-4b53-94a8-028c5c60b858/concepts/2f928913-21f6-4611-9055-01744acc344f

https://github.com/jeremy-shannon/CarND-Advanced-Lane-Lines/blob/master/project.ipynb

https://github.com/ksakmann/CarND-Advanced-Lane-Lines


### Distortion and warping

In [ ]:
filename = 'test_images/test1.jpg'
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_size = (img.shape[1], img.shape[0])

mtx, dist = camera_calibration(img_size)

### Auxiliary: Extract frames from video

In [ ]:
import os
from moviepy.editor import *

def extract_frames(movie, times, imgdir, image):
    clip = VideoFileClip(movie)
    for i, t in enumerate(times):
        # imgpath = os.path.join(imgdir, '{}.png'.format(t))
        imgpath = os.path.join(imgdir, 'harder_challenge{}.jpg'.format(i))
        clip.save_frame(imgpath, t)

#videofile = "test_videos/project_video.mp4"
#videofile = "test_videos/challenge_video.mp4"
videofile = "test_videos/harder_challenge_video.mp4"
imagefile = 'test7'
#times = [1., 3., 6., 10., 13., 15.]
times = [5., 9., 12., 14.8, 17., 23., 26., 29., 34., 36., 39., 42.]

extract_frames(videofile, times, 'test_images', imagefile)

### Gradient

In [ ]:
images = glob.glob('test_images/*.jpg')
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    mag_binary = gradient_magnitude_threshold(img, sobel_kernel=9, thresh=(30, 100))
    dir_binary = gradient_direction_threshold(img, sobel_kernel=15, thresh_deg=(35., 75.))
    x_binary   = gradient_x_threshold(img, sobel_kernel=9, thresh=(20, 100))

    combined = np.zeros_like(dir_binary)
    combined[((mag_binary == 1) & (dir_binary == 1))] = 1
    
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(mag_binary, cmap='gray')
    ax2.set_title('Thresholded Gradient Magnitude', fontsize=20)
    ax3.imshow(dir_binary, cmap='gray')
    ax3.set_title('Thresholded Gradient Direction', fontsize=20)
    ax4.imshow(x_binary, cmap='gray')
    ax4.set_title('X', fontsize=20)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### Colors

In [ ]:
# images = glob.glob('test_images/test*.jpg')
# images = glob.glob('test_images/challenge*.jpg')
images = glob.glob('test_images/harder_challenge*.jpg')
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    color_select(img)

### Pipeline test on single images

In [ ]:
# Load all test images
# images = glob.glob('test_images/*.jpg')
images = ['test_images/straight_lines1.jpg']
#mages = ['test_images/test8.jpg']
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Test only the image processing pipeline
    if 1:
        undistorted, processed, M, Minv = image_processing_pipeline(img, mtx, dist)
        
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(img)
        ax1.set_title('Original', fontsize=20)

        ax2.imshow(processed, cmap='gray')
        ax2.set_title('Processed', fontsize=20)
        ax2.grid(True)
        
    # Test the whole pipeline
    if 0:
        left_line = Line()
        right_line = Line()
        output = pipeline(img)
        f, ax = plt.subplots()
        ax.imshow(output)
        

### Pipeline test on video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip

filename = "test_videos/project_video.mp4"
# filename = "test_videos/challenge_video.mp4"
# filename = "test_videos/harder_challenge_video.mp4"
clip = VideoFileClip(filename)
white_output = filename[:-4] + '_output.mp4'

# Video boundaries
start_second = 18
end_second = 27

# Variables used by function "pipeline"
left_line = Line()
right_line = Line()
white_clip = clip.fl_image(pipeline)#.subclip(start_second,end_second)

%time white_clip.write_videofile(white_output, audio=False)